# RISS에서 서지정보 자동 내보내기

## 00 라이브러리

In [ ]:
# 필요 라이브러리 설치
!pip install pandas selenium tqdm beautifulsoup4 requests

# 라이브러리 로드
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import os
import shutil
import requests
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import UnexpectedAlertPresentException

#### 다운로드 폴더 설정(riss 폴더)

In [ ]:
# 원하는 다운로드 절대경로 설정가능(예:'C:\\Downloads')
download_path = os.getcwd()+'\\riss'
download_path

In [ ]:
# 참고 : https://cozynow.tistory.com/43
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
  "download.default_directory": download_path,
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})

## 01 크롬 가동 및 셀레늄 시험
아래 링크에서 운영체제에 맞는 크롬 드라이버 다운로드
https://chromedriver.chromium.org/downloads

In [ ]:
driver = webdriver.Chrome("chromedriver.exe", chrome_options=options)

#### 논문 검색조건
출판년도 : 2000~2019  
학과정보 : '국어국문' OR '국문'  
학위 : 국내박사

In [ ]:
driver.get('http://www.riss.kr/search/Search.do?isDetailSearch=Y&searchGubun=true&viewYn=OP&strQuery=&queryText=&exQuery=&resultSearch=false&icate=bib_t&order=%2FDESC&strSort=DATE&pageScale=100&isTab=Y&sflag=1&fsearchMethod=searchDetail&isFDetailSearch=N&fsearchSort=&fsearchOrder=&resultKeyword=&pageNumber=1&field1=znMajor&keyword1=%EA%B5%AD%EC%96%B4%EA%B5%AD%EB%AC%B8&op1=OR&field2=znMajor&keyword2=%EA%B5%AD%EB%AC%B8&op2=AND&field3=znSubject&keyword3=&op3=AND&field4=znAbstract&keyword4=&op4=AND&field5=znKtoc&keyword5=&colName=bib_t&p_year1=2000&p_year2=2019&mat_subtype=T2')

In [ ]:
# 파일선택
driver.find_element_by_css_selector('#divContent > div.rightContent > div > div.srchResultW > div:nth-child(1) > div.resultTop1 > label > span').click()

In [ ]:
# 내보내기
driver.find_element_by_css_selector('#divContent > div.rightContent > div > div.srchResultW > div:nth-child(1) > div.resultTop1 > ul > li:nth-child(1) > a').click()

In [ ]:
# 팝업창 컨트롤
driver.switch_to.window(driver.window_handles[-1])

In [ ]:
# Excel 저장 선택
driver.find_element_by_css_selector('#wrap > form > div > div.popCont > div:nth-child(1) > div > ul > li:nth-child(3) > label').click()

In [ ]:
# Excel 상세정보 선택
driver.find_element_by_css_selector('#riss_gubun > ul > li:nth-child(2) > label').click()

In [ ]:
# 데이터 표시양식 '가로' 설정
driver.find_element_by_css_selector('#excel_gubun > ul > li:nth-child(2) > label').click()

In [ ]:
# 내보내기 클릭
driver.find_element_by_css_selector('#riss_gubun > div.btnBunch > a.btnType1').click()

In [ ]:
# 다시 메인 윈도우로 복귀
driver.switch_to.window(driver.window_handles[0])

## 02 자동 내보내기

#### RISS에서 가장 마지막 페이지 번호

In [ ]:
n = 227

#### 자동 다운로드 파일 이름 설정

In [ ]:
keyword = '국어국문'

#### 자동 내보내기 시작

In [ ]:
for list_num in tqdm(range(1,n+1)):
    # print(list_num)

    # 파일선택
    driver.implicitly_wait(10)
    driver.find_element_by_css_selector('#divContent > div.rightContent > div > div.srchResultW > div:nth-child(1) > div.resultTop1 > label > span').click()
    # 내보내기
    driver.implicitly_wait(60)
    driver.find_element_by_css_selector('#divContent > div.rightContent > div > div.srchResultW > div:nth-child(1) > div.resultTop1 > ul > li:nth-child(1) > a').click()
    # 팝업창 컨트롤
    driver.implicitly_wait(10)
    driver.switch_to.window(driver.window_handles[-1])
    # Excel 저장 선택
    driver.implicitly_wait(20)
    driver.find_element_by_css_selector('#wrap > form > div > div.popCont > div:nth-child(1) > div > ul > li:nth-child(3) > label').click()
    # Excel 상세정보 선택
    driver.implicitly_wait(10)
    driver.find_element_by_css_selector('#riss_gubun > ul > li:nth-child(2) > label').click()
    # 데이터 표시양식 '가로' 설정
    driver.implicitly_wait(10)
    driver.find_element_by_css_selector('#excel_gubun > ul > li:nth-child(2) > label').click()
    # 내보내기 클릭
    driver.implicitly_wait(10)
    driver.find_element_by_css_selector('#riss_gubun > div.btnBunch > a.btnType1').click()
    
    # 다운로드 완료될 때까지 대기
    driver.implicitly_wait(10)
    try:
        filename = max([download_path + '\\' + f for f in os.listdir(download_path) if f=='myCabinetExcelData.xls'],key=os.path.getctime)
    except ValueError as e:
        time.sleep(1)

    # 팝업창 닫기
    driver.implicitly_wait(5)
    driver.close()
    # 다시 메인 화면으로 스위치
    driver.implicitly_wait(5)
    driver.switch_to.window(driver.window_handles[0])

    # 방금 다운 받은 파일 이름 바꾸기 -> 중복방지
    time.sleep(2)
    try:
        filename = max([download_path + '\\' + f for f in os.listdir(download_path) if f=='myCabinetExcelData.xls'],key=os.path.getctime)
        time.sleep(0.5)
        shutil.move(filename,os.path.join(download_path,f"{keyword}_{list_num}.xls")) ## 파일 이름 + num 설정
    except ValueError:
        time.sleep(0.5)
    except FileNotFoundError:
        time.sleep(1)

    # 다음 페이지 이동
    time.sleep(1)
    if list_num!=n:
        if (list_num)%10!=0:
            for i in range(10):
            # 다음페이지 클릭
                try:
                    driver.implicitly_wait(10)
                    if(list_num+1)%10!=0:
                        # driver.find_element_by_css_selector(f'#level4_mainContent > form > div.tabWarp.page > div.tabCont > div > div.passing > span.num > a:nth-child({(list_num+1)%10})').click()
                        
                        # 10페이지 이하 일때
                        if list_num<10:
                            driver.find_element_by_css_selector(f'#divContent > div.rightContent > div > div.paging > a:nth-child({((list_num+1)%10)+1})').click()
                        else:
                            driver.find_element_by_css_selector(f'#divContent > div.rightContent > div > div.paging > a:nth-child({((list_num+1)%10)+2})').click()
                    else:
                        # driver.find_element_by_css_selector(f'#level4_mainContent > form > div.tabWarp.page > div.tabCont > div > div.passing > span.num > a:nth-child(10)').click()
                        if list_num<10:
                            driver.find_element_by_css_selector(f'#divContent > div.rightContent > div > div.paging > a:nth-child(11)').click()
                        else:
                            driver.find_element_by_css_selector(f'#divContent > div.rightContent > div > div.paging > a:nth-child(12)').click()
                except NoSuchElementException as e:
                    print('retry in 1s.')
                    time.sleep(1)
                else:
                    break
        else:
            # 현재 10 * n 페이지 일때는 다음 화살표 클릭
            driver.implicitly_wait(10)
            #if list_num<=10:
                # driver.find_element_by_css_selector('#level4_mainContent > form > div.tabWarp.page > div.tabCont > div > div.passing > span:nth-child(3) > a > img').click()
            driver.find_element_by_css_selector('#divContent > div.rightContent > div > div.paging > a.next1').click()
            print('\n next tab')
            # else:
            #     # driver.find_element_by_css_selector('#level4_mainContent > form > div.tabWarp.page > div.tabCont > div > div.passing > span:nth-child(4) > a > img').click()
            #     driver.find_element_by_css_selector('#divContent > div.rightContent > div > div.paging > a.next2').click()
            #     print('next tab')
            #list_num += 1